In [1]:
#Desabilita logs e mantém apenas logs críticos (para evitar o libcuda ficar me avisando que a GPU tá funcionando)
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [2]:
import pandas as pd
import numpy as np 
import seaborn as sb
import matplotlib.pyplot as plt
import scipy as spy
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
df = pd.read_csv('../datasets/winequality-white.csv')

2021-08-16 10:07:52.437029: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Esse dataset já está tratado e não tem valores N/A:

In [3]:
pd.isna(df).sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

E todos os valores já estão como float, fora a qualidade que está como valor inteiro:

In [5]:
df['quality'].value_counts()

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64

<h2>Separando o dataset em dados de treinamento e teste </h2>

In [15]:
X =  df.drop(columns=["baixa", "media", "alta"], axis=1)
y=  df[['baixa','media','alta']]

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, shuffle=True)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3589, 11)
(898, 11)
(3589, 3)
(898, 3)


<h1>Treinamento do dataset </h1>

In [16]:
dimension = X_train.shape[1]
from keras import backend as K
def create_model():
    model = Sequential()
    model.add(layers.Dense(10, input_dim=dimension, activation='relu'))
    model.add(layers.Dense(1, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 33        
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


2021-08-16 10:07:57.410958: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-16 10:07:57.446213: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-16 10:07:57.446531: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0c:00.0 name: NVIDIA GeForce GTX 660 Ti computeCapability: 3.0
coreClock: 1.0715GHz coreCount: 7 deviceMemorySize: 1.95GiB deviceMemoryBandwidth: 134.29GiB/s
2021-08-16 10:07:57.446563: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-16 10:07:57.448593: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-16 10:07:57.448656: I tensorflow/stream_executor/p

In [17]:
history=model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=20, batch_size=10)

Epoch 1/20


2021-08-16 10:07:57.507938: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-16 10:07:57.508232: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3593305000 Hz


359/359 [==============================] - 0s 752us/step - loss: 6.3494 - accuracy: 0.7278 - val_loss: 0.5355 - val_accuracy: 0.8853
Epoch 2/20
359/359 [==============================] - 0s 486us/step - loss: 0.4715 - accuracy: 0.9050 - val_loss: 0.4358 - val_accuracy: 0.9143
Epoch 3/20
359/359 [==============================] - 0s 484us/step - loss: 0.3845 - accuracy: 0.9175 - val_loss: 0.3521 - val_accuracy: 0.9243
Epoch 4/20
359/359 [==============================] - 0s 477us/step - loss: 0.3238 - accuracy: 0.9301 - val_loss: 0.3816 - val_accuracy: 0.9098
Epoch 5/20
359/359 [==============================] - 0s 507us/step - loss: 0.3005 - accuracy: 0.9351 - val_loss: 0.3130 - val_accuracy: 0.9321
Epoch 6/20
359/359 [==============================] - 0s 461us/step - loss: 0.2909 - accuracy: 0.9356 - val_loss: 0.2860 - val_accuracy: 0.9343
Epoch 7/20
359/359 [==============================] - 0s 488us/step - loss: 0.2869 - accuracy: 0.9365 - val_loss: 0.2840 - val_accuracy: 0.9343
Epo

A função de ativação `softmax`dev

In [19]:
test_mse, test_mae = model.evaluate(X_test, y_test, verbose=1)
print('Mean Squared Error: ',test_mse, '\n','Mean Absolute Error: ',test_mae)

29/29 [==============================] - 0s 346us/step - loss: 0.2815 - accuracy: 0.9332
Mean Squared Error:  0.2814875543117523 
 Mean Absolute Error:  0.9331848621368408
